# Calificación y Rúbrica Final: Prueba Técnica FastAPI

**Candidato:** [Nombre del Candidato]
**Fecha:** [Fecha de Evaluación]

## Resumen de la Evaluación

El código presentado demuestra un entendimiento fundamental de **FastAPI** y **Pydantic**. El candidato ha logrado implementar los `endpoints` CRUD y conectarlos a una base de datos SQLite, utilizando un archivo `functions.py` para encapsular algunas operaciones de la base de datos y el parseo de resultados. Sin embargo, se identifican áreas críticas de mejora en la arquitectura, el manejo de la base de datos y la gestión de errores, lo que sitúa la solución en un **nivel básico** en el contexto de una aplicación de producción.

La principal desviación del requisito fue el uso de una base de datos persistente en lugar de una lista en memoria, lo cual, aunque demuestra un conocimiento adicional, no cumplió con la especificación original. Este es un punto importante a considerar en la evaluación.

---

## Rúbrica de Evaluación Detallada

### 1. Funcionalidad y Cumplimiento de Requisitos (Puntaje: 17/60)

| Criterio | Puntos Obtenidos / Puntos Máximos | Comentarios |
| :--- | :---: | :--- |
| **Implementación CRUD** | 7 / 20 | Los `endpoints` existen, pero la lógica para `POST`, `PUT`, y `DELETE` en `main.py` contiene errores significativos (uso incorrecto de `fetchone()`, falta de `raise` para `HTTPException`). tampoco usa una implementación de tipo ORM - todo el crud no es funcional |
| **Manejo de la Base de Datos** | 5 / 20 | El uso de `sqlite3` directo en `main.py` es ineficiente y propenso a errores. El archivo `functions.py` ayuda a encapsular la creación de la DB y el parseo, pero la gestión de conexiones y cursores sigue siendo manual y repetitiva en `main.py`. No se sigue la especificación de usar una lista en memoria. |
| **Aseguramiento de los datos** | 5 / 20 | No aplica the operations on the key concepts extracted from the first chapter of "The Lean Startup" (the provided text). por ende no existen datos para manipular - aunque cumple con el shema de la aplicación |

### 2. Calidad de Código y Estilo (Puntaje: 5/30)

| Criterio | Puntos Obtenidos / Puntos Máximos | Comentarios |
| :--- | :---: | :--- |
| **Estructura y Arquitectura** | 0 / 15 | Aunque existe `functions.py`, la arquitectura sigue siendo de bajo nivel. La lógica de la API, los modelos y las operaciones de la base de datos (más allá de `parseo` y `creación`) están fuertemente acoplados en `main.py`. No hay una separación clara de capas (p.ej., servicios, repositorios). |
| **Manejo de Errores** | 5 / 15 | Se usan `HTTPException`, lo que es correcto, pero la implementación es defectuosa (`status_code=200` y falta de `raise`). Los `endpoints` `PUT` y `DELETE` no devuelven respuestas adecuadas ni manejan correctamente la ausencia de recursos, tampoco utiliza validadores de entrada ni salida de recursos, no utiliza documentación de los errores y su codigo no se encuentra definido en un paradigma ni estructural ni de datos |

### 3. Documentación y Buenas Prácticas (Puntaje: 5/30)

| Criterio | Puntos Obtenidos / Puntos Máximos | Comentarios |
| :--- | :---: | :--- |
| **Legibilidad y Estilo (PEP 8)** | 5 / 15 | El código es funcional pero carece de consistencia en el estilo. No utiliza **tipos de datos** (`typing`) en los parámetros de las funciones. La función `parse_sqlite` es útil, pero el retorno condicional (`if len(tuples) > 1: return structured_data else: return structured_data[0]`) puede ser confuso y es mejor que el `endpoint` decida cómo presentar un solo resultado. |
| **Documentación** | 0 / 15 | No hay Docstrings ni comentarios explicativos significativos. La documentación automática de FastAPI no se enriquece con descripciones o ejemplos. |

---

## Retroalimentación Detallada y Código Refactorizado (Ejemplo de Nivel Intermedio)

A continuación, se muestra un ejemplo de cómo se podría **refactorizar** el código para mejorar su arquitectura y seguir las mejores prácticas. 

### 1. Refactorización: Separación de Responsabilidades y Capa de Datos

Se separa la lógica de la base de datos en su propio módulo, `crud.py` (o `repository.py`), que interactúa con SQLite. El `main.py` se centra únicamente en las rutas de la API y la inyección de dependencias. La funcionalidad de `parse_sqlite` se integra directamente en la capa de datos o se maneja con `row_factory` de `sqlite3`.

```python
# crud.py (Capa de Operaciones de Base de Datos)
import sqlite3
from typing import List, Optional, Dict, Any
from pydantic import BaseModel

DATABASE_PATH = "book.sqlite3"

class ConceptCreate(BaseModel):
    name: str
    definition: str
    book_section: str

class ConceptInDB(ConceptCreate):
    id: int

def get_db_connection():
    conn = sqlite3.connect(DATABASE_PATH)
    conn.row_factory = sqlite3.Row  # Permite acceder a las columnas por nombre
    return conn

def create_concepts_table():
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute('''
    CREATE TABLE IF NOT EXISTS concepts (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        name TEXT NOT NULL,
        definition TEXT NOT NULL,
        book_section TEXT NOT NULL
    )
    ''')
    conn.commit()
    conn.close()

def get_all_concepts() -> List[ConceptInDB]:
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM concepts")
    concepts = [ConceptInDB(**dict(row)) for row in cursor.fetchall()]
    conn.close()
    return concepts

def create_concept(concept: ConceptCreate) -> Optional[ConceptInDB]:
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute(
        "INSERT INTO concepts (name, definition, book_section) VALUES (?, ?, ?)",
        (concept.name, concept.definition, concept.book_section)
    )
    conn.commit()
    concept_id = cursor.lastrowid
    conn.close()
    if concept_id:
        return get_concept_by_id(concept_id) # Recupera el concepto completo con ID
    return None

def get_concept_by_id(concept_id: int) -> Optional[ConceptInDB]:
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT * FROM concepts WHERE id = ?", (concept_id,))
    row = cursor.fetchone()
    conn.close()
    if row:
        return ConceptInDB(**dict(row))
    return None

def update_concept(concept_id: int, concept: ConceptCreate) -> Optional[ConceptInDB]:
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute(
        "UPDATE concepts SET name = ?, definition = ?, book_section = ? WHERE id = ?",
        (concept.name, concept.definition, concept.book_section, concept_id)
    )
    conn.commit()
    rows_affected = cursor.rowcount
    conn.close()
    if rows_affected > 0:
        return get_concept_by_id(concept_id)
    return None

def delete_concept(concept_id: int) -> bool:
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("DELETE FROM concepts WHERE id = ?", (concept_id,))
    conn.commit()
    rows_affected = cursor.rowcount
    conn.close()
    return rows_affected > 0

# Inicializar la base de datos si no existe
create_concepts_table()
```

### 2. Refactorización: Lógica de la API y Manejo de Errores con Inyección de Dependencias

Ahora el `main.py` es más limpio, usa los tipos de datos de forma explícita y maneja los errores correctamente, utilizando las funciones de `crud.py`.

```python
# main.py (Refactorizado)
from fastapi import FastAPI, HTTPException, Depends, status
from typing import List
import crud # Importamos las funciones de la capa de datos
from crud import ConceptCreate, ConceptInDB # Importamos los modelos de Pydantic

app = FastAPI(title="Lean Startup Concepts API", description="API RESTful para gestionar conceptos de 'The Lean Startup'.")

# Dependencia para la base de datos (si se usara un ORM o una sesión de DB)
# En este caso simple con SQLite, las funciones de crud.py manejan la conexión.

@app.get(
    "/concepts", 
    response_model=List[ConceptInDB], 
    summary="Obtener todos los conceptos",
    description="Retorna una lista de todos los conceptos almacenados."
)
def get_concepts_endpoint():
    return crud.get_all_concepts()

@app.post(
    "/concepts", 
    response_model=ConceptInDB, 
    status_code=status.HTTP_201_CREATED, 
    summary="Crear un nuevo concepto",
    description="Permite crear un nuevo concepto y lo añade a la base de datos."
)
def create_concept_endpoint(concept: ConceptCreate):
    new_concept = crud.create_concept(concept)
    if not new_concept:
        raise HTTPException(status_code=status.HTTP_500_INTERNAL_SERVER_ERROR, detail="No se pudo crear el concepto.")
    return new_concept

@app.get(
    "/concepts/{concept_id}", 
    response_model=ConceptInDB, 
    summary="Obtener un concepto por ID",
    description="Retorna un concepto específico utilizando su ID. Si no existe, devuelve un error 404."
)
def get_concept_endpoint(concept_id: int):
    concept = crud.get_concept_by_id(concept_id)
    if not concept:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Concepto no encontrado.")
    return concept

@app.put(
    "/concepts/{concept_id}", 
    response_model=ConceptInDB, 
    summary="Actualizar un concepto existente",
    description="Actualiza un concepto existente por su ID. Si no existe, devuelve un error 404."
)
def update_concept_endpoint(concept_id: int, concept: ConceptCreate):
    updated_concept = crud.update_concept(concept_id, concept)
    if not updated_concept:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Concepto no encontrado para actualizar.")
    return updated_concept

@app.delete(
    "/concepts/{concept_id}", 
    status_code=status.HTTP_204_NO_CONTENT, 
    summary="Eliminar un concepto",
    description="Elimina un concepto por su ID. Si no existe, devuelve un error 404."
)
def delete_concept_endpoint(concept_id: int):
    success = crud.delete_concept(concept_id)
    if not success:
        raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="Concepto no encontrado para eliminar.")
    return # No content for 204

```

## Recomendaciones Finales

Para mejorar, se recomienda al candidato:

1.  **Seguir las Especificaciones**: Es crucial adherirse a los requisitos del ejercicio. Si se opta por una solución diferente (como una DB en lugar de memoria), se debe justificar. 
2.  **Manejo de la Base de Datos**: Aunque `functions.py` es un buen paso, la gestión de conexiones y cursores aún es muy manual. Considerar el uso de un **ORM** como **SQLAlchemy** (con `SQLModel` para integración con Pydantic) para una gestión más segura y abstracta de la base de datos. 
3.  **Arquitectura por Capas**: Implementar una arquitectura más estructurada con capas de **repositorio/CRUD** y **servicios** para desacoplar la lógica de negocio de las rutas de la API. 
4.  **Inyección de Dependencias**: Utilizar `Depends` de FastAPI para gestionar las conexiones a la base de datos o instancias de clases de servicio. 
5.  **Manejo de Errores Robusto**: Centralizar la gestión de errores con `exception_handler` y utilizar los códigos de estado HTTP de `status` de FastAPI para mayor claridad. 
6.  **Tipado y Documentación**: Hacer uso extensivo de **tipos de datos** (`typing`) y **Docstrings** para mejorar la legibilidad y mantenibilidad del código. 

---

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Datos de la evaluación: puntajes obtenidos y puntajes máximos
criterios = [
    "Funcionalidad y Cumplimiento de Requisitos",
    "Calidad de Código y Estilo",
    "Documentación y Buenas Prácticas"
]

puntos_obtenidos = [17, 5, 5]
puntos_maximos = [60, 30, 30]

# Calcular el porcentaje de cada criterio
porcentajes = [(obtenido / maximo) * 100 for obtenido, maximo in zip(puntos_obtenidos, puntos_maximos)]

# Número de variables (criterios)
num_variables = len(criterios)

# Crear una lista de ángulos para cada eje del gráfico
angulos = np.linspace(0, 2 * np.pi, num_variables, endpoint=False).tolist()

# El gráfico debe ser circular, así que se duplica el primer valor
porcentajes += porcentajes[:1]
angulos += angulos[:1]

# Iniciar la figura del gráfico
fig, ax = plt.subplots(figsize=(8, 8), subplot_kw=dict(polar=True))

# Dibujar las líneas del gráfico de araña
ax.fill(angulos, porcentajes, color='red', alpha=0.25)
ax.plot(angulos, porcentajes, color='red', linewidth=2, linestyle='solid', label='Evaluación')

# Establecer las etiquetas y los ticks
ax.set_theta_offset(np.pi / 2)
ax.set_theta_direction(-1)
ax.set_xticks(angulos[:-1])
ax.set_xticklabels(criterios, fontsize=12, weight='bold')

# Establecer los límites del eje radial y las etiquetas
ax.set_ylim(0, 100)
ax.set_yticks([20, 40, 60, 80, 100])
ax.set_yticklabels(["20", "40", "60", "80", "100%"], color="grey", size=8)

# Añadir un título y una leyenda
plt.title('Fortalezas de la Evaluación del Código', size=16, color='blue', y=1.1)
ax.legend(loc='upper right', bbox_to_anchor=(0.1, 0.1))

plt.show()

ModuleNotFoundError: No module named 'matplotlib'